<a href="https://colab.research.google.com/github/dr-kinder/playground/blob/dev/dolfinx_boundary_conditions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
try:
    # Import FEniCSx libraries for finite element analysis.
    import dolfinx
except ImportError:
    # If they are not found, install them.  Then import them.
    !wget "https://fem-on-colab.github.io/releases/fenicsx-install-complex.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

In [3]:
dolfinx.__version__

'0.6.0.0'

In [33]:
import numpy as np
import dolfinx
import ufl
from mpi4py import MPI
from dolfinx.io import XDMFFile
import petsc4py.PETSc as petsc


p = 2
k0 = np.pi
theta = 0
mesh = dolfinx.mesh.create_box(MPI.COMM_WORLD, [[0, 0, 0], [4, 4, 4]], [16, 16, 16])
n = ufl.FacetNormal(mesh)

# Definition of function space
# element = ufl.FiniteElement("N1curl", ufl.tetrahedron, p)
# V = dolfinx.fem.FunctionSpace(mesh, element)

V = dolfinx.fem.VectorFunctionSpace(mesh, ("Lagrange", 2))

In [34]:
def incoming_wave(x):
    d = np.cos(theta) * x[0] + np.sin(theta) * x[1]
    out = np.zeros(x.shape, dtype=np.complex128)
    out[2] = np.exp(1.0j * k0 * d)
    return out

# Incoming wave
Ei = dolfinx.fem.Function(V)
Ei.interpolate(incoming_wave)
g = ufl.cross(ufl.curl(Ei), n) + 1j * k0 * ufl.cross(n, ufl.cross(ufl.curl(Ei), n))
h =  ufl.dot(ufl.grad(Ei), n) + 1j * k0 * Ei

In [35]:
# Define variational problem
E = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# # Weak Form
a  = ufl.inner(ufl.curl(E), ufl.curl(v)) * ufl.dx 
a -= k0**2 * ufl.inner(E, v) * ufl.dx
a += 1j * k0 * ufl.inner(ufl.cross(n, E), ufl.cross(n, v)) * ufl.ds

# L = ufl.inner(h, v) * ufl.ds
L = ufl.inner(g, v) * ufl.ds
petsc_options = {"ksp_type": "preonly", "pc_type": "lu"}

In [36]:
u = dolfinx.fem.Function(V)

In [37]:
solver = dolfinx.fem.petsc.LinearProblem(a, L, [], u, petsc_options)

In [ ]:
solver.solve()

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "out.xdmf", "w") as file:
    file.write_mesh(mesh)
    file.write_function(u)

In [ ]:
from IPython import display
import pyvista

# Start a virtual plot window for PyVista in CoLab.
pyvista.start_xvfb()
pyvista.set_jupyter_backend("pythreejs")

In [ ]:
# Prepare the mesh for plotting.
topology, cells, geometry = dolfinx.plot.create_vtk_mesh(mesh)

# Turn the mesh into a PyVista grid.
grid = pyvista.UnstructuredGrid(topology, cells, geometry)

# Create the plot and export it to HTML.
plotter = pyvista.Plotter(window_size=(800, 400))
renderer = plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

# Save the HTML file.
plotter.export_html("./grid.html", backend="pythreejs")

# Use the IPython library to embed the HTML in the CoLab notebook.
display.HTML(filename='/content/grid.html')

In [ ]:
pyvista.set_jupyter_backend("pythreejs")
topology, cells, geometry = dolfinx.plot.create_vtk_mesh(V)
values = u.x.array.real.reshape((geometry.shape[0], len(u)))

# Create a point cloud of glyphs
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)
function_grid["u"] = values
glyphs = function_grid.glyph(orient="u", factor=0.05)

# Create a pyvista-grid for the mesh
grid = pyvista.UnstructuredGrid(*dolfinx.plot.create_vtk_mesh(mesh, mesh.topology.dim))

# Create plotter
plotter = pyvista.Plotter(window_size=(800, 400))
plotter.add_mesh(grid, style="wireframe", color="k")
plotter.add_mesh(glyphs)
plotter.view_xy()


# Save the HTML file.
plotter.export_html("./grid.html", backend="pythreejs")

# Use the IPython library to embed the HTML in the CoLab notebook.
display.HTML(filename='/content/grid.html')
